In [8]:
# Import the relevant libraries
from __future__ import division
import pandas as pd
import json
import time
from datetime import datetime, timedelta
import os
import folium
import csv

In [31]:
# working on a one file data
folder="./data/crowdsource-internal/"

# msisdn ='6588769918'
log1 ='6598194630_20170213_010123.log'#for exploring only

log2 ='6588769918_20170210_220100.log'


In [32]:
f = open(os.path.join(folder,log1))
data1=json.load(f)
f.close()

In [11]:
f = open(os.path.join(folder,log2))
data2=json.load(f)
f.close()

In [72]:
data_key =data1.keys()
print data_key

data_key.remove('locations')

print data_key

meta_data = data_key[2:]
print meta_data

set(meta_data) in set(data_key)

[u'msisdn', u'locations', u'appver', u'osver', u'model', u'trips', u'manufacturer']
[u'msisdn', u'appver', u'osver', u'model', u'trips', u'manufacturer']
[u'osver', u'model', u'trips', u'manufacturer']


False

In [53]:
data1['osver']

u'5.0.1'

In [35]:
data1['locations'][:2]

[{u'acc': u'14.142',
  u'cid': u'244622337',
  u'lat': u'1.2987251',
  u'lon': u'103.8463769',
  u'ts': u'1486945334232'},
 {u'acc': u'116.1',
  u'cid': u'244622337',
  u'lat': u'1.4671031',
  u'lon': u'103.7686717',
  u'ts': u'1486945370999'}]

In [15]:
# Preprosessing the epoch time
def convert_epoch_time(t):
    # Convert epoch time to utc time
    # PARAMS: t is the epoch time either in seconds or milliseconds. Type : str
    # Output : t in datetime.datetime format


    if len(t)> 12: # t is in milliseconds
        date_time = time.gmtime(int(t)/1000)
    else:
        date_time = time.gmtime(int(t))
        
    # convert to datetime.datetime format
    return datetime(*date_time[:6])
    



In [16]:
# Testing to convert epoch time
x='1486756802106' #in milliseconds

print convert_epoch_time(x)


2017-02-10 20:00:02


In [17]:
df = pd.DataFrame()

In [51]:
df_details1 = pd.DataFrame.from_records(data1['locations'],index='ts')
#print df_details[:4]
# Convert the index of the dataframe to datetime.datetime format

df_details1.index = df_details1.index.map(convert_epoch_time)

df_details1[:5]



,acc,cid,lat,lon
2017-02-13 00:22:14,14.142,244622337,1.2987251,103.8463769
2017-02-13 00:22:50,116.1,244622337,1.4671031,103.7686717
2017-02-13 00:22:56,1299.999,244622337,1.2950771,103.8453569
2017-02-13 00:23:45,68.4,244622337,1.3006329,103.839338
2017-02-13 00:23:55,38.79,244622337,1.3005276,103.8393826


In [66]:
#Produce the dict for the other relevant metadata
metadata_dict={}
for x in meta_data:
#     print x
#     print data1[x]
#     print "-"*20
    df_details1[x] = str(data1[x])
    

    

In [67]:
df_details1[:5]

,acc,cid,lat,lon,appver,osver,model,trips,manufacturer
2017-02-13 00:22:14,14.142,244622337,1.2987251,103.8463769,1.0,5.0.1,GT-I9506,[],samsung
2017-02-13 00:22:50,116.1,244622337,1.4671031,103.7686717,1.0,5.0.1,GT-I9506,[],samsung
2017-02-13 00:22:56,1299.999,244622337,1.2950771,103.8453569,1.0,5.0.1,GT-I9506,[],samsung
2017-02-13 00:23:45,68.4,244622337,1.3006329,103.839338,1.0,5.0.1,GT-I9506,[],samsung
2017-02-13 00:23:55,38.79,244622337,1.3005276,103.8393826,1.0,5.0.1,GT-I9506,[],samsung


In [68]:
# Insert the columns for the other meta_data
meta_ser = pd.Series(metadata_dict)

df_sample["appver"] =1.0


In [19]:
# combine the two frames
df = df.append(df_details1)
df[:5]

,acc,cid,lat,lon,signal
2017-02-10 20:00:02,22.817,20483393,1.4042991,103.9053817,-93
2017-02-10 20:00:05,46.0,20483393,1.4041914,103.9054802,-93
2017-02-10 20:00:22,22.824,20483393,1.4042991,103.9053817,-93
2017-02-10 20:00:24,30.0,20483393,1.4041913,103.90548,-93
2017-02-10 20:00:42,22.84,20483393,1.4042991,103.9053817,-93


In [20]:
# add the other columns to the dataframe


In [21]:
df_details2 = pd.DataFrame.from_records(data2['locations'],index='ts')
#print df_details[:4]
# Convert the index of the dataframe to datetime.datetime format

df_details2.index = df_details2.index.map(convert_epoch_time)

df_details2[:5]

,acc,cid,lat,lon,signal
2017-02-10 21:00:05,22.816,20483393,1.4042991,103.9053817,-93
2017-02-10 21:00:15,58.0,20483393,1.4040958,103.9055678,-93
2017-02-10 21:00:19,22.821,20483393,1.4042991,103.9053817,-93
2017-02-10 21:00:32,63.0,20483393,1.4040994,103.9055713,-93
2017-02-10 21:00:35,22.823,20483393,1.4042991,103.9053817,-93


In [22]:
#combine both the dataframes
df  = df.append(df_details2)

df[:5]

,acc,cid,lat,lon,signal
2017-02-10 20:00:02,22.817,20483393,1.4042991,103.9053817,-93
2017-02-10 20:00:05,46.0,20483393,1.4041914,103.9054802,-93
2017-02-10 20:00:22,22.824,20483393,1.4042991,103.9053817,-93
2017-02-10 20:00:24,30.0,20483393,1.4041913,103.90548,-93
2017-02-10 20:00:42,22.84,20483393,1.4042991,103.9053817,-93


In [37]:
for x in data_key[2:]:
    print x
    print data1[x]
    print '_______'

appver
1.0
_______
osver
5.0.1
_______
model
GT-I9506
_______
trips
[]
_______
manufacturer
samsung
_______


In [24]:
folder ='./data/crowdsource-internal/'
files = os.listdir(folder)
files = [x for x in files if x.endswith('.log')]

In [73]:
x=['6598282016', '6598636442']
y=['6596642145', '65111111', '650000000000009', '6591390516', '65999', '6591378121', '6596480975', '6596309066', '6598194630', '6591148900', '6598282016', '650544249111', '6598636442', '6594515139', '6581817105', '6590176722', '65123', '6588769918']

In [79]:
set(x).issubset(y)

True